Content Based:- Based On Similarity Of Content.
    
Collaborative Filtering Based:- Based Of Interest Of Users.

# Content Based Recommendation System

In [214]:
import os
print(os.listdir())

['.ipynb_checkpoints', 'movie-recommendation-system-cosine-similarity-1.ipynb', 'tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']


# Import Libraries

In [215]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [216]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [217]:
print(len(movies.columns))
movies.head(1)

20


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [218]:
print(len(credits.columns))
credits.head(1)

4


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [219]:
movies_credits = movies.merge(credits,on='title')
print(len(movies_credits.columns))
movies_credits.head(1)

23


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [220]:
for i,j in zip(movies_credits['id'],movies_credits['movie_id']):
    if(i!=j):
        print(i,j,"Got it")

72710 1255 Got it
1255 72710 Got it
268 2661 Got it
2661 268 Got it
39269 10844 Got it
10844 39269 Got it


In [221]:
movies_credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [222]:
print(movies_credits.shape)

(4809, 23)


In [223]:
movies_credits.isnull().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

In [224]:
# movies_credits.isna().sum()

In [225]:
# Features in our system:- genres, id, keywards, title, overview, cast, crew.

movies_credits = movies_credits[['movie_id','title','genres','cast','crew','keywords','overview']]
movies_credits.head(1)

,movie_id,title,genres,cast,crew,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


# Data Processing and Text Cleaning

In [226]:
movies_credits.isnull().sum()
# print(movies_credits.isna().sum())

movie_id    0
title       0
genres      0
cast        0
crew        0
keywords    0
overview    3
dtype: int64

In [227]:
movies_credits.isnull().sum().sum()

3

In [228]:
movies_credits = movies_credits.dropna()

In [229]:
movies_credits.isnull().sum()

movie_id    0
title       0
genres      0
cast        0
crew        0
keywords    0
overview    0
dtype: int64

In [230]:
movies_credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4806 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4806 non-null   int64 
 1   title     4806 non-null   object
 2   genres    4806 non-null   object
 3   cast      4806 non-null   object
 4   crew      4806 non-null   object
 5   keywords  4806 non-null   object
 6   overview  4806 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.4+ KB


In [231]:
movies_credits.duplicated().sum()

0

In [232]:
movies_credits=movies_credits.drop_duplicates()
movies_credits.head(1)

,movie_id,title,genres,cast,crew,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


In [233]:
movies_credits.iloc[0]

movie_id                                                19995
title                                                  Avatar
genres      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
cast        [{"cast_id": 242, "character": "Jake Sully", "...
crew        [{"credit_id": "52fe48009251416c750aca23", "de...
keywords    [{"id": 1463, "name": "culture clash"}, {"id":...
overview    In the 22nd century, a paraplegic Marine is di...
Name: 0, dtype: object

# Working Phase

## **Geners**

In [234]:
movies_credits['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [235]:
## taking genre

import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
        break
    return L

In [236]:
movies_credits['genres'] = movies_credits['genres'].apply(convert)

In [237]:
movies_credits['genres']

0            [Action]
1         [Adventure]
2            [Action]
3            [Action]
4            [Action]
            ...      
4804         [Action]
4805         [Comedy]
4806         [Comedy]
4807               []
4808    [Documentary]
Name: genres, Length: 4806, dtype: object

# **crew**

In [238]:
movies_credits['crew']

0       [{"credit_id": "52fe48009251416c750aca23", "de...
1       [{"credit_id": "52fe4232c3a36847f800b579", "de...
2       [{"credit_id": "54805967c3a36829b5002c41", "de...
3       [{"credit_id": "52fe4781c3a36847f81398c3", "de...
4       [{"credit_id": "52fe479ac3a36847f813eaa3", "de...
                              ...                        
4804    [{"credit_id": "52fe44eec3a36847f80b280b", "de...
4805    [{"credit_id": "52fe487dc3a368484e0fb013", "de...
4806    [{"credit_id": "52fe4df3c3a36847f8275ecf", "de...
4807    [{"credit_id": "52fe4ad9c3a368484e16a36b", "de...
4808    [{"credit_id": "58ce021b9251415a390165d9", "de...
Name: crew, Length: 4806, dtype: object

In [239]:
## taking director from crew

import ast
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [240]:
movies_credits['crew'] = movies_credits['crew'].apply(fetch_director)

In [241]:
movies_credits['crew']

0           [James Cameron]
1          [Gore Verbinski]
2              [Sam Mendes]
3       [Christopher Nolan]
4          [Andrew Stanton]
               ...         
4804     [Robert Rodriguez]
4805         [Edward Burns]
4806          [Scott Smith]
4807          [Daniel Hsia]
4808     [Brian Herzlinger]
Name: crew, Length: 4806, dtype: object

## **cast**

In [242]:
movies_credits['cast']

0       [{"cast_id": 242, "character": "Jake Sully", "...
1       [{"cast_id": 4, "character": "Captain Jack Spa...
2       [{"cast_id": 1, "character": "James Bond", "cr...
3       [{"cast_id": 2, "character": "Bruce Wayne / Ba...
4       [{"cast_id": 5, "character": "John Carter", "c...
                              ...                        
4804    [{"cast_id": 1, "character": "El Mariachi", "c...
4805    [{"cast_id": 1, "character": "Buzzy", "credit_...
4806    [{"cast_id": 8, "character": "Oliver O\u2019To...
4807    [{"cast_id": 3, "character": "Sam", "credit_id...
4808    [{"cast_id": 3, "character": "Herself", "credi...
Name: cast, Length: 4806, dtype: object

In [243]:
## taking top three actors

def cast(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter = counter+1
        else:
            break
    return L

In [244]:
movies_credits['cast'] = movies_credits['cast'].apply(cast)

In [245]:
movies_credits['cast']

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4806, dtype: object

## **keywords**

In [246]:
movies_credits['keywords']

0       [{"id": 1463, "name": "culture clash"}, {"id":...
1       [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2       [{"id": 470, "name": "spy"}, {"id": 818, "name...
3       [{"id": 849, "name": "dc comics"}, {"id": 853,...
4       [{"id": 818, "name": "based on novel"}, {"id":...
                              ...                        
4804    [{"id": 5616, "name": "united states\u2013mexi...
4805                                                   []
4806    [{"id": 248, "name": "date"}, {"id": 699, "nam...
4807                                                   []
4808    [{"id": 1523, "name": "obsession"}, {"id": 224...
Name: keywords, Length: 4806, dtype: object

In [247]:
# taking keywords

def keywords_function(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [248]:
movies_credits['keywords'] = movies_credits['keywords'].apply(keywords_function)

In [249]:
movies_credits['keywords'] 

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4806, dtype: object

## **overview**

In [250]:
movies_credits['overview'] 

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4804    El Mariachi just wants to play his guitar and ...
4805    A newlywed couple's honeymoon is upended by th...
4806    "Signed, Sealed, Delivered" introduces a dedic...
4807    When ambitious New York attorney Sam is sent t...
4808    Ever since the second grade when he first saw ...
Name: overview, Length: 4806, dtype: object

In [251]:
movies_credits['overview']  = movies_credits['overview'].apply(lambda x: x.split())

In [252]:
movies_credits['overview']

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4804    [El, Mariachi, just, wants, to, play, his, gui...
4805    [A, newlywed, couple's, honeymoon, is, upended...
4806    ["Signed,, Sealed,, Delivered", introduces, a,...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Ever, since, the, second, grade, when, he, fi...
Name: overview, Length: 4806, dtype: object

In [253]:
movies_credits.head(1)

,movie_id,title,genres,cast,crew,keywords,overview
0,19995,Avatar,[Action],"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin..."


In [254]:
movies_credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4806 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4806 non-null   int64 
 1   title     4806 non-null   object
 2   genres    4806 non-null   object
 3   cast      4806 non-null   object
 4   crew      4806 non-null   object
 5   keywords  4806 non-null   object
 6   overview  4806 non-null   object
dtypes: int64(1), object(6)
memory usage: 429.4+ KB


In [255]:
# Remove spaces

movies_credits['crew'] = movies_credits['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies_credits['genres'] = movies_credits['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies_credits['keywords'] = movies_credits['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies_credits['cast'] = movies_credits['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [256]:
movies_credits.head(1)

,movie_id,title,genres,cast,crew,keywords,overview
0,19995,Avatar,[Action],"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin..."


In [257]:
movies_credits['tags'] = movies_credits['overview']+movies_credits['genres']+movies_credits['keywords']+movies_credits['cast']+movies_credits['crew']

In [258]:
# converting tages into string

movies_credits['tags'] = movies_credits['tags'].apply(lambda x: " ".join(x))
movies_credits['tags'] = movies_credits['tags'].apply(lambda x: x.lower())

In [259]:
resulting_movies_credits = movies_credits[['movie_id','title','tags']]

In [260]:
resulting_movies_credits.head(2)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


In [261]:
resulting_movies_credits['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

## **Text Processing Deep Dive**

In [263]:
import nltk
from nltk.corpus import stopwords

In [264]:
# remove stopwords from tags

resulting_movies_credits['tags'] = resulting_movies_credits['tags'].apply(lambda x: x.join(i for i in x.split() if i not in stopwords.words('english')))

In [265]:
resulting_movies_credits['tags'][0]

'22ndin the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameroncentury,in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameronparaplegicin the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following or

In [292]:
resulting_movies_credits.head(1)

,movie_id,title,tags
0,19995,Avatar,"22ndin the 22nd century, a parapleg marin is d..."


In [268]:
from nltk.stem import PorterStemmer

print(PorterStemmer().stem('boxed'))
print(PorterStemmer().stem('boxes'))

box
box


In [269]:
resulting_movies_credits['tags'] = resulting_movies_credits['tags'] .apply(lambda x: " ".join([PorterStemmer().stem(i) for i in x.split()]))

In [273]:
resulting_movies_credits.head(1)

,movie_id,title,tags
0,19995,Avatar,"22ndin the 22nd century, a parapleg marin is d..."


## **Bags Of Words**

In [317]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(max_features=5000)
vectors = cv.fit_transform(resulting_movies_credits['tags']).toarray()

In [318]:
print(vectors.shape)

(4806, 5000)


## **Apply Cosine Similarity**

In [319]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
print(similarity.shape)

(4806, 4806)


In [320]:
# similarity[0]

In [321]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1089, 0.4038594976582577),
 (1216, 0.39301659657305726),
 (507, 0.38380084006118026),
 (2409, 0.3837711202915073),
 (3608, 0.38238303229768167)]

In [322]:
resulting_movies_credits.head(1)

,movie_id,title,tags
0,19995,Avatar,"22ndin the 22nd century, a parapleg marin is d..."


## **Recommendation System**

In [332]:
def RecommendationEngine(movie_name):
    temp = resulting_movies_credits['title'] == movie_name
    if(temp.sum()!=1):
        print('No Such Movie Found !')
        return
    movie_index = resulting_movies_credits[resulting_movies_credits['title'] == movie_name].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True,key=lambda x:x[1])[1:10]
    
    for i in movie_list:
        print(resulting_movies_credits.iloc[i[0]].title)

In [333]:
RecommendationEngine('Avater')

No Such Movie Found !


In [334]:
RecommendationEngine('Avatar')

Aliens in the Attic
Aliens vs Predator: Requiem
Independence Day
Aliens
Apollo 18
Brave New Girl
Tears of the Sun
Avengers: Age of Ultron
Men in Black II


In [335]:
RecommendationEngine('Independence Day')

Mission: Impossible - Ghost Protocol
The SpongeBob Movie: Sponge Out of Water
The Monkey King 2
Guiana 1838
The Baader Meinhof Complex
Aliens vs Predator: Requiem
Iraq for Sale: The War Profiteers
Gladiator
Men in Black


In [336]:
RecommendationEngine('Independaence Day')

No Such Movie Found !
